In [ ]:
%load_ext kedro.extras.extensions.ipython

In [ ]:
%reload_kedro

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

### read catalog

In [ ]:
catalog.list()

In [ ]:
pd.read_parquet

In [ ]:
df = context.catalog.load('transactions')

In [ ]:
df = context.catalog.load('transactions_sample')

In [ ]:
df.loc[:, df.columns.isin({'article_id', 'price', 'customer_id', 'nla'})]

In [ ]:
df

In [ ]:
pd.concat([df[df.sales_channel_id>1], df[df.sales_channel_id==1].sample(n=1_000)], axis=0)ss

In [ ]:
df.head()

In [ ]:
try:
    pd.to_datetime(df.loc[:, 'customdsaer_id'])
except KeyError:
    print('Given date_column does not exist in df')
    raise
except ValueError:
    print('Given date_column is not convertible to datetime')
    raise
except:
    print('lol')

In [ ]:
pd.to_datetime(df.loc[:, 'custodmer_id'])

In [ ]:
df = context.catalog.load('transactions')

df_articles = context.catalog.load('articles')

In [ ]:
df_customers = context.catalog.load('customers')

### transactions

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.t_dat = pd.to_datetime(df.t_dat)

In [ ]:
df.dtypes

### Sample step 1: only latest transactions

In [ ]:
df.t_dat.describe(datetime_is_numeric=True)

In [ ]:
df[df.t_dat>'2020-04-01'].t_dat.describe(datetime_is_numeric=True)

In [ ]:
df = df[df.t_dat>'2020-04-01'].reset_index(drop=True)

In [ ]:
df.shape

### Sample step 2: stratified sampling

In [ ]:
df_customers

In [ ]:
cd_customers_trans = df.groupby(['customer_id'])['t_dat'].nunique().reset_index(name='no_transactions')

In [ ]:
cd_customers_trans.columns

In [ ]:
df_customers = df_customers.merge(cd_customers_trans, on='customer_id', how='left')

In [ ]:
df_customers.columns

### fillna() for sampling

In [ ]:
df_customers.isna().sum()

In [ ]:
df_customers['age'] = df_customers['age'].fillna(df_customers['age'].median())

In [ ]:
df_customers['no_transactions'] = df_customers['no_transactions'].fillna(0)

In [ ]:
df_customers[['age', 'no_transactions']].describe()

In [ ]:
df_customers['age_bin'] = pd.qcut(df_customers['age'], 4, labels=['1', '2', '3', '4']).astype(str)

In [ ]:
df_customers.no_transactions.hist()

#### i'm doing custom bins for no_transactions, because the distribution is heavily concentrated around 0

In [ ]:
cond_list = [
    df_customers['no_transactions']==0,
    df_customers['no_transactions']<=1,
    df_customers['no_transactions']<=3,
    df_customers['no_transactions']<=6,
    df_customers['no_transactions']<=9
]

In [ ]:
choice_list = [
    '1',
    '2',
    '3',
    '4',
    '5'
]

In [ ]:
np.unique(np.select(cond_list, choice_list, default='6'), return_counts=True)

In [ ]:
df_customers['no_transactions_bin'] = np.select(cond_list, choice_list, default='6')

In [ ]:
df_customers.shape

In [ ]:
_, df_customers_sample = train_test_split(df_customers, test_size=0.1,
                                          stratify=df_customers[['age_bin', 'no_transactions_bin']])

In [ ]:
df_customers_sample.shape

### check

In [ ]:
df_customers[['age_bin']].value_counts(normalize=True)

In [ ]:
df_customers[['no_transactions_bin']].value_counts(normalize=True)

In [ ]:
df_customers_sample[['age_bin']].value_counts(normalize=True)

In [ ]:
df_customers_sample[['no_transactions_bin']].value_counts(normalize=True)

### filter out other datasets

In [ ]:
df_customers_sample.drop(['no_transactions', 'age_bin', 'no_transactions_bin'], axis=1, inplace=True)

In [ ]:
df_customers_sample

In [ ]:
unique_customer_id_set = set(df_customers_sample.customer_id.unique())

In [ ]:
df_sample = df[df.customer_id.isin(unique_customer_id_set)]

In [ ]:
### i don't whether to sample ARTICLES -> if filtered off then no cold-start problem.
### On the other hand, you NEED TO SAMPLE for EMBEDDINGS, maybe another pipeline for that?

In [ ]:
df_sample.shape, df_customers_sample.shape

### Tests

In [ ]:
transactions = context.catalog.load('transactions_sample')

In [ ]:
articles = context.catalog.load('articles_sample')

In [ ]:
customers = context.catalog.load('customers_sample')

In [ ]:
transactions['t_dat'].min(), transactions['t_dat'].max()

In [ ]:
transactions['article_id'].nunique()

In [ ]:
articles['article_id'].nunique()

In [ ]:
transactions['customer_id'].nunique()

In [ ]:
customers['customer_id'].nunique()

In [ ]:
transactions.shape

In [ ]:
transactions.merge(articles['article_id'], on='article_id').shape

In [ ]:
transactions.merge(customers['customer_id'], on='customer_id').shape